In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
import os
import time

Using TensorFlow backend.


In [2]:
X_serial = np.load('../data/processed_data/m4015_session2/X_serial.npy')
y_serial = np.load('../data/processed_data/m4015_session2/y_serial.npy')

res = tf.one_hot(y_serial, depth=2)
with tf.Session() as sess:
    y_serial = (sess.run(res))

In [3]:
batch_size = 128
lookback = 200


def generator(X, y, batch_size, lookback):
    batch_features = np.zeros((batch_size, lookback, 1))
    batch_labels = np.zeros((batch_size, lookback, 2))
    while True:
        for i in range(batch_size):
            while True:
                index = np.random.randint(0, X.shape[0]-lookback)
                if np.logical_and(np.sum(y[index:index+lookback][:,1]) / lookback > 0.3,
                                  np.sum(y[index:index+lookback][:,1]) / lookback < 0.7):
                    break
            batch_features[i] = X[index:index+lookback]
            batch_labels[i] = y[index:index+lookback]
#         return batch_features, batch_labels
        yield batch_features, batch_labels

def save_model(model, save_dir='models'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    t = time.strftime("%Y%m%d-%H:%M:%S")
    path = os.path.join(save_dir, t+'.h5')  
    model.save(path)

In [4]:
# _, y = generator(X_serial[:1000000,:], y_serial[:1000000,:], batch_size, lookback)
# y[10, :, 1]

In [5]:
train_generator = generator(X_serial[:1000000,:], y_serial[:1000000], batch_size, lookback)
validate_generator = generator(X_serial[1000000:,:], y_serial[1000000:], batch_size, lookback)

In [6]:
n_dim = 1
input_shape = (None,n_dim)
batch_size = lookback

keras.backend.clear_session()
inputs = keras.layers.Input(shape=input_shape)

x = keras.layers.LSTM(100, activation='tanh', return_sequences=True)(inputs)
predictions = keras.layers.Dense(2, activation='softmax')(x)

model = keras.models.Model(inputs=inputs, outputs=predictions)
model.compile(loss=['categorical_crossentropy'],
              optimizer=keras.optimizers.Adam(lr=0.01))


print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 1)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 100)         40800     
_________________________________________________________________
dense_1 (Dense)              (None, None, 2)           202       
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
epochs = 20

# history = model.fit(train_input, train_output, batch_size=batch_size, epochs=epochs, 
#                     verbose=True, validation_data=(test_input, test_output))

history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=10, 
                              verbose=1,
                              validation_data=validate_generator,
                              validation_steps=100)

save_model(model)

Epoch 1/10
100/100 [==============================] - 106s 1s/step - loss: 0.6955 - val_loss: 0.6967
Epoch 2/10
100/100 [==============================] - 94s 939ms/step - loss: 0.6912 - val_loss: 0.6944
Epoch 3/10
100/100 [==============================] - 94s 935ms/step - loss: 0.6896 - val_loss: 0.6849
Epoch 4/10
100/100 [==============================] - 98s 980ms/step - loss: 0.6928 - val_loss: 0.7005
Epoch 5/10
100/100 [==============================] - 96s 963ms/step - loss: 0.6944 - val_loss: 0.6941
Epoch 6/10
100/100 [==============================] - 100s 1s/step - loss: 0.6920 - val_loss: 0.6936
Epoch 7/10
100/100 [==============================] - 96s 963ms/step - loss: 0.6898 - val_loss: 0.6915
Epoch 8/10
100/100 [==============================] - 98s 977ms/step - loss: 0.6906 - val_loss: 0.6943
Epoch 9/10
100/100 [==============================] - 93s 929ms/step - loss: 0.6909 - val_loss: 0.6952
Epoch 10/10
100/100 [==============================] - 94s 940ms/step - loss:

In [8]:
pwd

'/Users/astasik/Desktop/ANN_Ripple_Detection/ANNs'

In [9]:
X_serial = np.load('../data/processed_data/m4015_session2/X_serial.npy')
y_serial = np.load('../data/processed_data/m4015_session2/y_serial.npy')

In [10]:
np.sum(y_serial)/len(y_serial)

0.020211432474913946